# 🚀 Master Notebook – Pipeline Spectroscopie DR5

Ce notebook orchestre le **workflow complet** pour entraîner un classifieur à partir des spectres LAMOST DR5 :  
**sélection d’un lot → génération/enrichissement du catalogue → prétraitement + features → entraînement → journaux & artefacts.**

**Sommaire rapide**
- [🧪 Étape 0 : SETUP & IMPORTS](#etape-0)
- [▶️ Lancer une session complète](#run-full)
- [1) Téléchargement des spectres](#step-1-download)
- [2) Sélection du lot de spectres](#step-2-select)
- [3) Catalogue : génération & enrichissement Gaia](#step-3-catalog)
- [3bis) Traitement & extraction des features](#step-3bis-features)
- [4) Entraînement du modèle](#step-4-train)

> Orchestrateur utilisé : **`MasterPipeline`**  
> Méthodes clés : `select_batch`, `generate_and_enrich_catalog`, `process_data`, `run_training_session`, `run_full_pipeline`, `interactive_training_runner`.

<a id="etape-0"></a>

#

## 🧪 Étape 0 : SETUP & IMPORTS

Initialise l’environnement, crée/valide l’arborescence des répertoires
(`RAW_DATA_DIR`, `CATALOG_DIR`, `PROCESSED_DIR`, `MODELS_DIR`, `REPORTS_DIR`)
et instancie **`MasterPipeline`**.

**Attendu après exécution :**
- un objet `pipeline` prêt à l’emploi,
- messages sur la racine du projet et, si configuré, tentative de connexion à **Gaia**.

> ℹ️ **UI interactive d’entraînement** : disponible avec `pipeline.interactive_training_runner()`.

In [ ]:
from utils import setup_project_env, load_env_vars
from pipeline.master import MasterPipeline
from astroquery.gaia import Gaia
from pipeline.classifier import SpectralClassifier

# Initialisation automatique de l'environnement et des chemins
paths = setup_project_env()

# Chargement des credentials Gaia depuis .env
env_vars = load_env_vars()

try:
    print("Tentative de connexion à l'archive Gaia...")
    Gaia.login(user=env_vars.get("GAIA_USER"), password=env_vars.get("GAIA_PASS"))
    print("Connexion à Gaia réussie.")
except Exception as e:
    print(f"AVERTISSEMENT : Échec de la connexion à Gaia ({e}). Le mode 'bulk' pourrait échouer.")

# Instanciation du pipeline maître
pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("\nSetup terminé. Tu es prêt à lancer ton pipeline.")

#

---

<a id="run-full"></a>
## ▶️ Lancer une session complète

Lance **tout le pipeline A→Z** :
`select_batch → generate_and_enrich_catalog → process_data → run_training_session`.

> 💡 **Paramètres conseillés** au début : `batch_size=200–500`, `n_estimators=200–400` (RF/XGB).  
> Active `enrich_gaia=True` lorsque la connectivité est stable.

In [ ]:
pipeline.run_full_pipeline(
    batch_size=500,                 # taille du lot
    model_type="RandomForest",      # "RandomForest" ou "XGBoost"
    n_estimators=100,               # arbres du modèle final
    prediction_target="main_class", # ex.: "main_class", "sub_class_top25", "sub_class_bins"
    save_and_log=True,              # sauvegarde modèle + rapport JSON
    enrich_gaia=False,              # True pour activer Gaia
    # ...kwargs Gaia si enrich_gaia=True
)

#

---

<a id="step-1-download"></a>
## 1) Téléchargement des spectres

Utilisation du script **`dr5_downloader.py`** encapsulé.  
Cette étape est externalisée dans **[01_download_spectra.ipynb](./01_download_spectra.ipynb)** (à exécuter au besoin).

> ⚠️ **Quota / temps** : selon le volume demandé, le téléchargement peut être long.

#

<a id="step-2-select"></a>
## 2) Sélection du lot de spectres

Choisit un **nouveau lot** de fichiers `.fits.gz` à traiter sans réutiliser de spectres déjà journalisés.

- `batch_size` : nombre de spectres,
- `strategy` : ex. `"random"`.

Le sélectionneur s’appuie sur **DatasetBuilder** pour garantir l’unicité des échantillons.


In [ ]:
pipeline.select_batch(batch_size=2500, strategy="random")

#

<a id="step-3-catalog"></a>
## 3) Catalogue : génération & enrichissement Gaia

À partir du lot courant, produit un **catalogue local** (CSV) et peut l’**enrichir via Gaia** (positions, photométrie…).

**Sorties :**
- `master_catalog_temp.csv` (catalogue local) puis `master_catalog_gaia.csv` si enrichi,
- mise à jour de `pipeline.master_catalog_df`.

> ℹ️ **Couplage Gaia** : géré par l’orchestrateur (appairage + stats).  
> ⚠️ **Connexion** : si l’authentification Gaia échoue, relance sans `enrich_gaia` ou vérifie tes identifiants.

In [ ]:
pipeline.generate_and_enrich_catalog(
    enrich_gaia=True,
    mode='bulk',
    include_risky=False,   # <-- active radius/mass/age -- beta donc en test
    ruwe_max=1.4           # optionnel: garde aussi les entrées à RUWE élevé - <1.4 est un bon filtre
)

#

<a id="step-3bis-features"></a>
## 3bis) Traitement & extraction des features

Exécute les **prétraitements spectraux** et l’**extraction de features**.  
Un CSV `features_YYYYMMDDTHHMMSSZ.csv` est écrit dans `processed/`.

- Met à jour `pipeline.features_df` (mémoire) & `pipeline.last_features_path` (disque).
- Features = mesures photométriques/astrométriques, indices de raies, résumés de voisinage spectral…

> 💡 Certaines étapes internes reposent sur des détections/associations de raies (Balmer, Ca II H/K, Mg_b, Na_D).

In [ ]:
pipeline.process_data()

#

<a id="step-4-train"></a>
## 4) Entraînement du modèle

Entraîne un **classifieur** (RF/XGBoost) avec **sélection de features** optionnelle (`SelectFromModel`, seuil `"median"` par défaut), puis **évalue** et **journalise**.

- Récap : nb de features conservées, scores, rapports, chemins des artefacts.
- En notebook : UI dédiée via `pipeline.interactive_training_runner()`.

> 💡 **Astuce** : commence avec RF pour un feedback rapide, puis passe à XGB pour gagner en performance.

In [ ]:
pipeline.interactive_training_runner()

#
#
#